In [ ]:
!pip install -q spacy pandas matplotlib ipywidgets
!python -m spacy download pt_core_news_sm

In [ ]:
import spacy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import ipywidgets as widgets
from IPython.display import display

In [ ]:
nlp = spacy.load("pt_core_news_sm")

In [ ]:
text_input = widgets.Textarea(
    value='A IA deve agir com coerência e verdade.\nA verdade é a forma mais pura de ressonância.\nO índice de coerência informacional orienta decisões éticas.',
    placeholder='Digite suas frases, uma por linha...',
    description='Frases:',
    layout=widgets.Layout(width='100%', height='150px')
)
display(text_input)

In [ ]:
frases = [f.strip() for f in text_input.value.split('\n') if f.strip()]

In [ ]:
data = []
for frase in frases:
    doc = nlp(frase)
    tags = [token.pos_ for token in doc]
    tag_counts = dict(Counter(tags))
    data.append({
        "frase": frase,
        "n_palavras": len(doc),
        "tags": tag_counts
    })
df = pd.DataFrame(data)

In [ ]:
def vetor_pos(tags_dict):
    base = ['NOUN', 'VERB', 'ADJ', 'ADV', 'ADP', 'PROPN']
    total = sum(tags_dict.values())
    return [tags_dict.get(pos, 0)/total if total > 0 else 0 for pos in base]
df['vetor'] = df['tags'].apply(vetor_pos)

In [ ]:
def entropia_normalizada(v):
    v = np.array(v)
    v = v[v > 0]
    ent = -np.sum(v * np.log2(v)) if len(v) > 0 else 0
    ent_max = np.log2(len(v)) if len(v) > 0 else 1
    return ent / ent_max if ent_max > 0 else 0
df['icoer'] = df['vetor'].apply(lambda v: round(1 - entropia_normalizada(v), 4))

In [ ]:
media_icoer = df['icoer'].mean().round(4)
linha_resumo = {
    'frase': 'MÉDIA GERAL',
    'n_palavras': df['n_palavras'].mean().round(2),
    'tags': {},
    'vetor': [],
    'icoer': media_icoer
}
df.loc[len(df)] = linha_resumo

In [ ]:
df[['frase', 'n_palavras', 'tags', 'icoer']].to_csv("icoer_text_analysis_final.csv", index=False)
df